In [140]:
import requests
import pandas as pd
import re
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df
import pickle
from pprint import pprint

In [145]:
links_pickle = open("URL.pickle","rb")
links = pickle.load(links_pickle)
print(links)

['https://www.sephora.com/product/luminous-silk-foundation-P393401?icid2=products%20grid:p393401', 'https://www.sephora.com/product/double-wear-stay-in-place-makeup-P378284?icid2=products%20grid:p378284', 'https://www.sephora.com/product/pro-filt-r-hydrating-longwear-foundation-P448702?icid2=products%20grid:p448702', 'https://www.sephora.com/product/pro-filtr-soft-matte-longwear-foundation-P87985432?icid2=products%20grid:p87985432', 'https://www.sephora.com/product/skin-fetish-sublime-perfection-foundation-P447519?icid2=products%20grid:p447519', 'https://www.sephora.com/product/your-skin-but-better-cc-cream-spf-50-P411885?icid2=products%20grid:p411885', 'https://www.sephora.com/product/ultra-hd-invisible-cover-foundation-P398321?icid2=products%20grid:p398321', 'https://www.sephora.com/product/tinted-moisturizer-broad-spectrum-P109936?icid2=products%20grid:p109936', 'https://www.sephora.com/product/teint-idole-ultra-24h-long-wear-foundation-P308201?icid2=products%20grid:p308201', 'https

In [148]:
prod_ids = []
for link in links: 
    prod_ids.append(re.findall('P\d{5,6}', url_test))

In [193]:
prod_ids

[['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P109927'],
 ['P10

In [214]:
sample_size = prod_ids[0:2]

In [215]:
sample_size

[['P109927'], ['P109927']]

In [210]:
# url = 'https://api.bazaarvoice.com/data/reviews.json'


# for prod in prod_ids:

#     params = {
#       'Filter':f'ProductId:{prod[0]}',
#       'Sort': 'TotalPositiveFeedbackCount:desc',
#       'Limit': 30,
#       'Offset': 0,
#       'Include': 'Products,Comments',
#       'Stats': 'Reviews',
#       'passkey': 'rwbw526r2e7spptqd2qzbkp7',
#       'apiversion': 5.4
#     }

#     reviews = []
#     loop = 0

#     while True:
#         params['Offset'] = len(reviews)
    
#         r = requests.get(url, params=params)
        
#         #break if error or exceed length of reviews
#         if (r.status_code != 200) or (len(reviews) >= r.json()['TotalResults']):
#             break

#         # add the list of results to current results
#         reviews.extend(r.json()['Results'])

#         # Give a pause, so we don't get blocked
#         time.sleep(0.5)

#         # Show how many reviews we scraped
# print(len(reviews))

In [216]:
url = 'https://api.bazaarvoice.com/data/reviews.json'


for prod in sample_size:

    params = {
      'Filter':f'ProductId:{prod[0]}',
      'Sort': 'TotalPositiveFeedbackCount:desc',
      'Limit': 30,
      'Offset': 0,
      'Include': 'Products,Comments',
      'Stats': 'Reviews',
      'passkey': 'rwbw526r2e7spptqd2qzbkp7',
      'apiversion': 5.4
    }

    reviews = []
    loop = 0

    while True:
        params['Offset'] = len(reviews)
    
        r = requests.get(url, params=params)
        
        #break if error or exceed length of reviews
        if (r.status_code != 200) or (len(reviews) >= r.json()['TotalResults']):
            break

        # add the list of results to current results
        reviews.extend(r.json()['Results'])

        # Give a pause, so we don't get blocked
        time.sleep(0.5)

        # Show how many reviews we scraped
print(len(reviews))

1406


In [465]:
review_df = pd.DataFrame(columns = [
                                    'ProductID',
                                    'UserNickname',
                                    'Rating',
                                    'Title',
                                    'ReviewText',
                                    'UserLocation',
                                    'Age',
                                    'SkinTone', 
                                    'SkinType',
                                    'Description'])

In [466]:
sample= reviews[0:6]

In [485]:
# for review in sample:
for i in range(len(sample)):
    table = {'ProductID': sample[i]['ProductId'],
             'UserNickname': sample[i]['UserNickname'],
             'Rating': sample[i]['Rating'],
             'Title':sample[i]['Title'],
             'ReviewText': sample[i]['ReviewText'],
             'UserLocation': sample[i]['UserLocation']}
    review_df = review_df.append(table, ignore_index=True)
    
    #age 
    try:
        review_df[i]['Age'] = sample[i]['ContextDataValues']['age']['Value']
    except KeyError:
        print("Erorr")
    
    #skinTone
    try:
        review_df[i]['SkinTone'] = sample[i]['ContextDataValues']['skinTone']['Value']
    except KeyError:
        print("Erorr")
    
    #skinType
    try:
        review_df[i]['SkinType'] = sample[i]['ContextDataValues']['skinType']['Value']
    except KeyError:
        print("Erorr")


Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr
Erorr


In [489]:
sample[1]['ContextDataValues']['age']['Value']

'45to54'

In [486]:
review_df

,ProductID,UserNickname,Rating,Title,ReviewText,UserLocation,Age,SkinTone,SkinType
0,P109927,SFValleymom,5,Easy on the eyes (and skin),"At 60, I am always looking for products that d...","North Hollywood, CA",NaN,NaN,NaN
1,P109927,ginger46,5,Buff and Wheat,This is an excellent cream eyeshadow/primer. I...,Washington,NaN,NaN,NaN
2,P109927,Stardusty,5,Excellent primer,I was hesitant to try this product but am so h...,"Strasburg, CO",NaN,NaN,NaN
3,P109927,prussia,5,Sensational Eye Shadow or Primer!,I first got a sample of this product in the co...,"Chicago , Il",NaN,NaN,NaN
4,P109927,hianna,5,Beautiful primer great for post-30 eyes,Many primers I've tried give my 50+ eyelids an...,"Maui, Hawaii",NaN,NaN,NaN
5,P109927,Maryam23,5,DOESN'T CREASE!,I haven't tried a lot of primers before this o...,None,NaN,NaN,NaN
6,P109927,SFValleymom,5,Easy on the eyes (and skin),"At 60, I am always looking for products that d...","North Hollywood, CA",NaN,NaN,NaN
7,P109927,ginger46,5,Buff and Wheat,This is an excellent cream eyeshadow/primer. I...,Washington,NaN,NaN,NaN
8,P109927,Stardusty,5,Excellent primer,I was hesitant to try this product but am so h...,"Strasburg, CO",NaN,NaN,NaN
9,P109927,prussia,5,Sensational Eye Shadow or Primer!,I first got a sample of this product in the co...,"Chicago , Il",NaN,NaN,NaN


In [454]:
sample[4]['ContextDataValues']['age']['Value']

'over54'

In [439]:
review_df

,ProductID,UserNickname,Rating,Title,ReviewText,UserLocation,Age,SkinTone,SkinType
0,P109927,SFValleymom,5,Easy on the eyes (and skin),"At 60, I am always looking for products that d...","North Hollywood, CA",NaN,NaN,NaN
1,P109927,ginger46,5,Buff and Wheat,This is an excellent cream eyeshadow/primer. I...,Washington,NaN,NaN,NaN
2,P109927,Stardusty,5,Excellent primer,I was hesitant to try this product but am so h...,"Strasburg, CO",NaN,NaN,NaN
3,P109927,prussia,5,Sensational Eye Shadow or Primer!,I first got a sample of this product in the co...,"Chicago , Il",NaN,NaN,NaN
4,P109927,hianna,5,Beautiful primer great for post-30 eyes,Many primers I've tried give my 50+ eyelids an...,"Maui, Hawaii",NaN,NaN,NaN
5,P109927,Maryam23,5,DOESN'T CREASE!,I haven't tried a lot of primers before this o...,None,NaN,NaN,NaN


In [141]:
pprint(reviews[0])

{'AdditionalFields': {},
 'AdditionalFieldsOrder': [],
 'AuthorId': '1331777113',
 'Badges': {'beautyInsider': {'BadgeType': 'Custom',
                              'ContentType': 'REVIEW',
                              'Id': 'beautyInsider'}},
 'BadgesOrder': ['beautyInsider'],
 'CampaignId': 'BV_RATING_SUMMARY',
 'ClientResponses': [],
 'CommentIds': [],
 'Cons': None,
 'ContentLocale': 'en_US',
 'ContextDataValues': {'IncentivizedReview': {'DimensionLabel': 'I received '
                                                                'this product '
                                                                'as a free '
                                                                'sample',
                                              'Id': 'IncentivizedReview',
                                              'Value': 'false',
                                              'ValueLabel': 'False'},
                       'StaffContext': {'DimensionLabel': 'I am a Sephora '
      